# Portfolio Builder using NSEPython and yFinance

This notebook dynamically fetches NSE equity symbols using the `nsepython` package, retrieves live stock fundamentals using `yfinance`, and builds an equal-weight portfolio based on user investment input.

---


In [39]:
# Import required libraries
from nsepython import nse_eq_symbols
import yfinance as yf
import pandas as pd
import math
import locale
import matplotlib.pyplot as plt
import plotly.express as px

## Step 1: Fetch NSE Equity Symbols

We use `nsepython` to fetch the full list of tradable NSE equity tickers.

Since there is no direct function to fetch Nifty 50 constituents, we fetch all symbols here and later select top 50 stocks by market capitalization.

In [3]:
# Fetch all NSE tradable equity symbols
all_symbols = nse_eq_symbols()
print(f"Total NSE Equity Symbols Fetched: {len(all_symbols)}")

# Append ".NS" suffix for yfinance tickers format
tickers = [symbol + ".NS" for symbol in all_symbols]

# Show a sample of 10 tickers
print("Sample tickers:", tickers[:10])

Total NSE Equity Symbols Fetched: 2176
Sample tickers: ['20MICRONS.NS', '21STCENMGM.NS', '360ONE.NS', '3IINFOLTD.NS', '3MINDIA.NS', '3PLAND.NS', '5PAISA.NS', '63MOONS.NS', 'A2ZINFRA.NS', 'AAATECH.NS']


## Step 2: Fetch Live Fundamentals from Yahoo Finance

Using yfinance, we get the latest stock price, market capitalization, P/E ratio, and dividend yield for all tickers.

**Note:** This step may take several minutes as it queries data for many stocks.


In [33]:
all_symbols = nse_eq_symbols()
tickers = [sym + ".NS" for sym in all_symbols[:500]]  
def fetch_batch_fundamentals(tickers):
    batch = yf.Tickers(' '.join(tickers))
    data = []
    for ticker in tickers:
        try:
            info = batch.tickers[ticker].info
            data.append({
                'Ticker': ticker,
                'Company Name': info.get('longName'),
                'Price': info.get('regularMarketPrice'),
                'Market Cap': info.get('marketCap'),
                'P/E Ratio': info.get('trailingPE'),
                'Dividend Yield': info.get('dividendYield')
            })
        except Exception as e:
            print(f"Failed to fetch {ticker}: {e}")
    return pd.DataFrame(data)

print("Fetching batch fundamentals...")
fundamentals_df = fetch_batch_fundamentals(tickers)

Fetching batch fundamentals...


In [34]:
fundamentals_df

,Ticker,Company Name,Price,Market Cap,P/E Ratio,Dividend Yield
0,20MICRONS.NS,20 Microns Limited,214.00,7.551312e+09,12.692763,0.57
1,21STCENMGM.NS,Twentyfirst Century Management Services Limited,51.78,5.501815e+08,None,4.15
2,360ONE.NS,360 One Wam Limited,1039.00,4.205461e+11,38.69646,1.17
3,3IINFOLTD.NS,3i Infotech Limited,21.71,3.888650e+09,9.52193,NaN
4,3MINDIA.NS,3M India Limited,29125.00,3.312241e+11,66.20221,0.55
...,...,...,...,...,...,...
495,DHARMAJ.NS,Dharmaj Crop Guard Limited,319.00,1.083822e+10,19.801365,NaN
496,DHRUV.NS,Dhruv Consultancy Services Limited,52.43,1.014843e+09,11.782023,0.76
497,DHUNINV.NS,Dhunseri Investments Limited,1433.00,8.737256e+09,8.980947,0.21
498,DIACABS.NS,Diamond Power Infrastructure Limited,151.00,7.967298e+10,209.72221,NaN


## Step 4: Select top 50 stocks by market cap

In [35]:
fundamentals_df.dropna(subset=['Market Cap'], inplace=True)
top_n = 10
portfolio_df = fundamentals_df.sort_values('Market Cap', ascending=False).head(top_n)

In [36]:
def style_portfolio_table(df):
    def highlight_pe(val):
        if pd.isna(val):
            return ''
        elif val < 15:
            return 'background-color: #c6efce'  
        elif val > 30:
            return 'background-color: #ffc7ce'  
        else:
            return ''
    def highlight_dividend(val):
        if pd.isna(val):
            return ''
        elif val > 0.02:
            return 'background-color: #c6efce'  # green for good yield
        else:
            return ''

    styled = df.style.format({
        'Price': "₹{:,.2f}",
        'Market Cap': "₹{:,.0f}",
        'Dividend Yield': "{:.2%}",
        'Total Value (INR)': "₹{:,.2f}"
    }).map(highlight_pe, subset=['P/E Ratio']).map(highlight_dividend, subset=['Dividend Yield']).set_caption('Styled Portfolio Table')
    
    return styled

In [37]:
styled_df = style_portfolio_table(portfolio_df)
print("Top 10 Stocks By Market Cap")
styled_df 

Top 10 Stocks By Market Cap


,Ticker,Company Name,Price,Market Cap,P/E Ratio,Dividend Yield
294,BHARTIARTL.NS,Bharti Airtel Limited,"₹1,869.50","₹11,203,220,865,024",31.880968,85.00%
238,BAJFINANCE.NS,Bajaj Finance Limited,₹987.35,"₹6,201,492,897,792",35.580180,44.00%
220,AXISBANK.NS,Axis Bank Limited,"₹1,161.40","₹3,603,683,606,528",12.662451,9.00%
231,BAJAJFINSV.NS,Bajaj Finserv Ltd.,"₹2,010.00","₹3,209,974,775,808",34.062023,5.00%
47,ADANIPORTS.NS,Adani Ports and Special Economic Zone Limited,"₹1,422.90","₹3,073,661,730,816",26.887756,50.00%
45,ADANIENT.NS,Adani Enterprises Limited,"₹2,594.30","₹2,994,291,081,216",46.112694,5.00%
275,BEL.NS,Bharat Electronics Limited,₹407.05,"₹2,975,445,549,056",53.842590,59.00%
48,ADANIPOWER.NS,Adani Power Limited,₹152.90,"₹2,948,629,790,720",23.201820,nan%
228,BAJAJ-AUTO.NS,Bajaj Auto Limited,"₹8,616.00","₹2,405,464,014,848",31.909927,242.00%
404,COALINDIA.NS,Coal India Limited,₹389.85,"₹2,402,539,798,528",7.244936,824.00%


In [52]:
## The Red Colour Represents Bad P/E
## The Green Colour Represents Good P/E

## Step 4: Input Total Investment and Calculate Shares to Buy

User inputs total cash available for investment.

Then, portfolio is allocated equally across the 50 stocks based on their latest prices to generate a shares-to-buy plan.

In [40]:
locale.setlocale(locale.LC_ALL, '')

def format_inr(value):
    if pd.isna(value):
        return "N/A"
    return f"₹{value:,.2f}"

def get_investment_amount():
    while True:
        try:
            user_input = input("Enter your total investment amount in INR (e.g. 1000000): ").replace(',', '').strip()
            total_investment = float(user_input)
            if total_investment <= 0:
                print("❌ Please enter a positive number.")
                continue
            return total_investment
        except ValueError:
            print("❌ Invalid input. Please enter a number (e.g. 1000000).")

def display_portfolio(portfolio_df, top_n=10):
    df = portfolio_df.copy()
    df['Total Value (INR)'] = df['Shares to Buy'] * df['Price']
    df['Price'] = df['Price'].apply(format_inr)
    df['Market Cap'] = df['Market Cap'].apply(format_inr)
    df['Total Value (INR)'] = df['Total Value (INR)'].apply(format_inr)

    print("\n✅ Your Portfolio Allocation:\n")
    print(df[['Ticker', 'Company Name', 'Price', 'Market Cap',
              'P/E Ratio', 'Dividend Yield', 'Shares to Buy', 'Total Value (INR)']].to_string(index=False))

def run_final_allocation(portfolio_df):
    total_investment = get_investment_amount()
    allocation_per_stock = total_investment / len(portfolio_df)
    portfolio_df['Shares to Buy'] = portfolio_df['Price'].apply(lambda p: math.floor(allocation_per_stock / p) if p else 0)
    display_portfolio(portfolio_df)

In [51]:
run_final_allocation(portfolio_df)

Enter your total investment amount in INR (e.g. 1000000):  265000



✅ Your Portfolio Allocation:

       Ticker                                  Company Name     Price             Market Cap  P/E Ratio  Dividend Yield  Shares to Buy Total Value (INR)
BHARTIARTL.NS                         Bharti Airtel Limited ₹1,869.50 ₹11,203,220,865,024.00  31.880968            0.85             14        ₹26,173.00
BAJFINANCE.NS                         Bajaj Finance Limited   ₹987.35  ₹6,201,492,897,792.00   35.58018            0.44             26        ₹25,671.10
  AXISBANK.NS                             Axis Bank Limited ₹1,161.40  ₹3,603,683,606,528.00  12.662451            0.09             22        ₹25,550.80
BAJAJFINSV.NS                            Bajaj Finserv Ltd. ₹2,010.00  ₹3,209,974,775,808.00  34.062023            0.05             13        ₹26,130.00
ADANIPORTS.NS Adani Ports and Special Economic Zone Limited ₹1,422.90  ₹3,073,661,730,816.00  26.887756            0.50             18        ₹25,612.20
  ADANIENT.NS                     Adani Enterprises